In [1]:
import pandas as pd
import folium
import json
import requests
from geopy import distance

In [2]:
df_seoul = pd.read_csv('../data/수정 후 데이터/seoul_stations.csv')

In [3]:
# 환승 가능한 역들
df_tran=[]
for i in df_seoul['지하철역'].unique():
    index = df_seoul[df_seoul['지하철역'] == i].index
    if len(df_seoul[df_seoul['지하철역'] == i]) > 1:
        for j in index:
            df_tran.append(df_seoul.iloc[j])
    else:
        pass

In [4]:
df_map = pd.concat([ df_seoul.iloc[:,:2], df_seoul.iloc[:,-3:] ], axis=1)
df_map.head(2)

,지하철역,호선명,주소,위도,경도
0,가산디지털단지,1호선,서울 금천구 가산동 468-4,37.480396,126.882662
1,개봉,1호선,서울 구로구 개봉동 415,37.494642,126.858716


In [5]:
#  01~09호선 분리
df_map_1 = df_map[df_map['호선명']=='1호선']
df_map_2 = df_map[df_map['호선명']=='2호선']
df_map_3 = df_map[df_map['호선명']=='3호선']
df_map_4 = df_map[df_map['호선명']=='4호선']
df_map_5 = df_map[df_map['호선명']=='5호선']
df_map_6 = df_map[df_map['호선명']=='6호선']
df_map_7 = df_map[df_map['호선명']=='7호선']
df_map_8 = df_map[df_map['호선명']=='8호선']
df_map_9 = df_map[df_map['호선명']=='9호선']
df_map_10 = df_map[df_map['호선명']=='경의선']
df_map_11 = df_map[df_map['호선명']=='경춘선']
df_map_12 = df_map[df_map['호선명']=='공항철도 1호선']
df_map_13 = df_map[df_map['호선명']=='분당선']
df_map_14 = df_map[df_map['호선명']=='우이신설선']
df_map_15 = df_map[df_map['호선명']=='중앙선']

In [6]:
df_map = df_map.groupby(['지하철역', '주소', '위도', '경도']).sum().reset_index()

In [7]:
df_map.loc[df_map[df_map['지하철역'] == '용산'].index, '호선명'] = '1호선중앙선'
df_map.loc[df_map[df_map['지하철역'] == '왕십리(성동구청)'].index, '호선명'] = '2호선5호선중앙선분당선'

In [8]:
df_tran = pd.concat([ pd.DataFrame(df_tran).iloc[:,:2], pd.DataFrame(df_tran).iloc[:,-3:] ], axis=1)

In [9]:
df_tran = df_tran.groupby(['지하철역', '주소','위도','경도']).sum().reset_index()
df_tran.head(2)

,지하철역,주소,위도,경도,호선명
0,가락시장,서울 송파구 가락동 184-23,37.493099,127.118263,3호선8호선
1,가산디지털단지,서울 금천구 가산동 468-4,37.480396,126.882662,1호선7호선


In [10]:
candidate = pd.read_csv('../data/수정 후 데이터/환승역 후보.csv')

In [11]:
candidate_location=[]
for i in range(0, len(candidate['지하철역'])):
    candidate_location = pd.concat([pd.DataFrame(candidate_location), df_map[ df_map['지하철역'] == candidate['지하철역'][i]] ], axis=0)

In [12]:
candidate_location = candidate_location.reset_index()
candidate_location.drop(columns='index', inplace=True)
candidate_location.head(2)

,지하철역,주소,위도,경도,호선명
0,강변(동서울터미널),서울 광진구 구의동 546-6,37.535118,127.094741,2호선
1,남부터미널(예술의전당),서울 서초구 서초동 1748-30,37.485200,127.016200,3호선


#### 1단계: 지도에 서울 내 지하철역 표시

In [13]:
map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

colors=['darkblue','darkgreen','darkorange','darkturquoise','slateblue','chocolate','olive',
      'deeppink','tan','mediumaquamarine','forestgreen','dodgeblue','gold','yellogreen','mediumaquamarine']


for i in range(1,16):
    for j in globals()[f'df_map_{i}'].index:
        folium.Circle(
        radius=200, 
        location=[globals()[f'df_map_{i}'].위도[j],globals()[f'df_map_{i}'].경도[j]],
        popup=folium.Popup(globals()[f'df_map_{i}'].호선명[j], max_width=200),
        tooltip=globals()[f'df_map_{i}'].지하철역[j],
        color=colors[i-1],
        fill=True,
        fill_color=colors[i-1]
    ).add_to(map)

for i in df_tran.index: 
    folium.Circle(
        radius=200, 
        location=[df_tran.위도[i],df_tran.경도[i]],
        popup=folium.Popup(df_tran.호선명[i], max_width=200),
        tooltip=df_tran.지하철역[i]+'(환승가능역)',
        color='gray',
        fill=True,
        fill_opacity=1,
        fill_color='darkgray'
    ).add_to(map)
    

    
title='<h3 align="center" style="font-size:30px">STEP 1: 현재 서울 내 운행 중인 지하철 표시</h3>'
map.get_root().html.add_child(folium.Element(title))

map

c:\Users\user\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


#### 2단계: 환승 후보역의 반경 2.3km 표시

In [14]:
## STEP 2 ##

map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

colors=['darkblue','darkgreen','darkorange','darkturquoise','slateblue','chocolate','olive',
      'deeppink','tan','mediumaquamarine','forestgreen','dodgeblue','gold','yellogreen','mediumaquamarine']

# 환승 후보역 지도에 표시
for i in candidate_location.index: 
    folium.Marker(
        radius=1000, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        icon = folium.Icon(
            color='purple',
            icon='train-subway',
            prefix='fa'
        )
    ).add_to(map)
for i in candidate_location.index: 
    folium.Circle(
        radius=2300, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        color='red',
        fill=True,
        fill_opacity=0.2,
        fill_color='red'
    ).add_to(map)
    
for i in range(1,16):
    for j in globals()[f'df_map_{i}'].index:
        folium.Circle(
        radius=200, 
        location=[globals()[f'df_map_{i}'].위도[j],globals()[f'df_map_{i}'].경도[j]],
        popup=folium.Popup(globals()[f'df_map_{i}'].호선명[j], max_width=200),
        tooltip=globals()[f'df_map_{i}'].지하철역[j],
        color=colors[i-1],
        fill=True,
        fill_color=colors[i-1]
    ).add_to(map)

for i in df_tran.index: 
    folium.Circle(
        radius=200, 
        location=[df_tran.위도[i],df_tran.경도[i]],
        popup=folium.Popup(df_tran.호선명[i], max_width=200),
        tooltip=df_tran.지하철역[i]+'(환승가능역)',
        color='gray',
        fill=True,
        fill_opacity=1,
        fill_color='darkgray'
    ).add_to(map)
    

    
title='<h3 align="center" style="font-size:30px">STEP 2: 환승 후보역 반경 2.3km 표시</h3>'
map.get_root().html.add_child(folium.Element(title))

map

#### 3단계: 환승 후보역 반경 2.3km 외 역 삭제

In [15]:
def creat_cand(can_gps, df_map, km):
    cand = []
    for i in df_map.index:
        map_gps = df_map.loc[i, '위도'],df_map.loc[i, '경도']
        dist = distance.distance(can_gps, map_gps).km
        if (dist < km) & (dist > 0):
            cand.append(df_map.loc[i])
    return pd.DataFrame(cand)

In [16]:
# 중간 산출물 저장(지도 그리기 용도)
ca = []
for i in candidate_location.index:
    km = 2.3
    can_gps = candidate_location.loc[i,'위도'], candidate_location.loc[i,'경도']
    ca.append(creat_cand(can_gps, df_map, km))

In [17]:
## STEP 3 ##

map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

# 환승 후보역 지도에 표시
for i in candidate_location.index: 
    folium.Marker(
        radius=1000, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        icon = folium.Icon(
            color='purple',
            icon='train-subway',
            prefix='fa'
        )
    ).add_to(map)
    
for i in candidate_location.index: 
    folium.Circle(
        radius=2300, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        color='red',
        fill=True,
        fill_opacity=0.2,
        fill_color='red'
    ).add_to(map)
    
for i in range(0,len(ca)):
    for j in ca[i].index:
        folium.Circle(
        radius=200, 
        location=[ca[i].위도[j],ca[i].경도[j]],
        popup=folium.Popup(ca[i].호선명[j], max_width=200),
        tooltip=ca[i].지하철역[j],
        color='yellow',
        fill=True,
        fill_color='yellow'
    ).add_to(map)
    
    
title='<h3 align="center" style="font-size:30px">STEP 3: 환승 후보역 반경 2.3km 외 역 삭제</h3>'
map.get_root().html.add_child(folium.Element(title))

map

#### 4단계: 2.3km 이내에 후보역과 다른 호선이면서 환승되지 않는 역 추출

In [18]:
def store_line(lines, line):
    if len(line) > 0:
        line = line.split('선')[0:-1]
        for li in line:
            li = li + '선'
            if li not in lines:
                lines.append(li)
    return lines

In [19]:
def store_lines(lines, candidate_station, cand):
    for i in cand.index:
        # 같은호선인 환승역들의 환승노선 저장
        if candidate_station['호선명'] in cand.loc[i, '호선명']:
            line = cand.loc[i, '호선명'].replace(candidate_station['호선명'], '')
            lines = store_line(lines, line)

        # 제거할 다른 노선 환승역의 환승노선 저장
        else:
            for li in lines:
                if li in cand.loc[i, '호선명']:
                    line = cand.loc[i, '호선명'].replace(li, '')
                    lines = store_line(lines, line)
    return lines

In [20]:
# 환승역인 역 제거
def remove_cand(cand, lines):
    for i in cand.index:
        if cand.loc[i, '호선명'].count('선') > 1:
            cand.drop(index=i, inplace=True)
        else:
            for line in lines:
                if line in cand.loc[i, '호선명']:
                    cand.drop(index=i, inplace=True)
                    break
    return cand

In [21]:
candidate_location_copy = candidate_location.copy()
candidate_name = []

In [22]:
# 2.3km 이내에 있는 역 중 조건 만족하는 역 추출
choice = pd.DataFrame(columns=['지하철역', '주소', '위도', '경도', '호선명', '후보역'])
for i in candidate_location.index:
    km = 2.3
    can_gps = candidate_location.loc[i,'위도'], candidate_location.loc[i,'경도']
    cand = creat_cand(can_gps, df_map, km)
    lines = [candidate_location.loc[i]['호선명']]
    candidate_station = candidate_location.loc[i]
    for j in range(3):
        lines = store_lines(lines, candidate_station, cand)
    cand = remove_cand(cand, lines)
    
    if len(cand) == 0:
        candidate_location_copy.drop(index=i, inplace=True)
    else:
        choice = pd.concat([choice, cand])
        for k in range(len(cand)):
            candidate_name.append(candidate_location.loc[i, '지하철역'])

In [23]:
choice['후보역'] = candidate_name

In [24]:
## STEP 4 ##

map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

# 환승 후보역 지도에 표시
for i in candidate_location.index: 
    folium.Marker(
        radius=1000, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        icon = folium.Icon(
            color='purple',
            icon='train-subway',
            prefix='fa'
        )
    ).add_to(map)
    
for i in candidate_location.index: 
    folium.Circle(
        radius=2300, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        color='red',
        fill=True,
        fill_opacity=0.2,
        fill_color='red'
    ).add_to(map)
    

for i in choice.index:
    folium.Circle(
    radius=200, 
    location=[choice.위도[i],choice.경도[i]],
    popup=folium.Popup(choice.호선명[i], max_width=200),
    tooltip=choice.지하철역[i],
    color='yellow',
    fill=True,
    fill_color='yellow'
).add_to(map)
    
    
title='<h3 align="center" style="font-size:25px">STEP 4: 2.3km 이내에 후보역과 다른 호선이면서 환승되지 않는 역 추출</h3>'
map.get_root().html.add_child(folium.Element(title))

map

c:\Users\user\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


#### 5단계: 4단계 이후 반경안에 역이 존재하는 후보군만 남기고 모두 제거

In [25]:
## STEP 5 ##

map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

# 환승 후보역 지도에 표시
for i in candidate_location_copy.index: 
    folium.Marker(
        radius=1000, 
        location=[candidate_location_copy.위도[i],candidate_location_copy.경도[i]],
        popup=folium.Popup(candidate_location_copy.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        icon = folium.Icon(
            color='purple',
            icon='train-subway',
            prefix='fa'
        )
    ).add_to(map)
    
for i in candidate_location_copy.index: 
    folium.Circle(
        radius=2300, 
        location=[candidate_location_copy.위도[i],candidate_location_copy.경도[i]],
        popup=folium.Popup(candidate_location_copy.호선명[i], max_width=200),
        tooltip=candidate_location_copy.지하철역[i]+'(환승후보역)',
        color='red',
        fill=True,
        fill_opacity=0.2,
        fill_color='red'
    ).add_to(map)
    

for i in choice.index:
    folium.Circle(
    radius=200, 
    location=[choice.위도[i],choice.경도[i]],
    popup=folium.Popup(choice.호선명[i], max_width=200),
    tooltip=choice.지하철역[i],
    color='yellow',
    fill=True,
    fill_color='yellow'
).add_to(map)
    
    
title='<h3 align="center" style="font-size:30px">STEP 5: 예비 후보역 추출 </h3>'
map.get_root().html.add_child(folium.Element(title))

map

#### 6단계: 최종 후보역 선택

In [26]:
# 인근 5개역 중 환승역과 같은 노선이 있으면 환승 후보역과 연결하는 역에서 탈락
for i in candidate_location_copy.index:
    station = candidate_location_copy.loc[i]
    cand_lines = choice[choice['후보역'] == station['지하철역']]['호선명'].unique()
    lines = []
    km = 5.75
    can_gps = station.loc['위도'], station.loc['경도']
    for j in df_map.index:
        map_gps = df_map.loc[j,'위도'], df_map.loc[j,'경도']
        dist = distance.distance(can_gps, map_gps).km
        if (dist < km) & (dist > 0) & (station['호선명'] in df_map.loc[j, '호선명']):
            for line in cand_lines:
                if line in df_map.loc[j, '호선명']:
                    for li in line.split('선')[:-1]:
                        if li not in station['호선명']:
                            lines.append(li + '선')
    if len(lines) > 0:
        for l in lines:
            index = choice[(choice['후보역'] == station['지하철역']) & (choice['호선명'] == l)].index
            choice.drop(index=index, inplace=True)

In [27]:
index = []
for i in choice['후보역'].unique():
    index.append(candidate_location_copy[candidate_location_copy['지하철역'] == i].index[0])
candidate_location_copy = candidate_location_copy.loc[index]

In [28]:
# 환승 후보역
candidate_location_copy

,지하철역,주소,위도,경도,호선명
0,강변(동서울터미널),서울 광진구 구의동 546-6,37.535118,127.094741,2호선
14,일원,서울 강남구 일원동 717,37.483989,127.084130,3호선
15,봉화산(서울의료원),서울 중랑구 신내동 643-1,37.617351,127.091360,6호선


In [29]:
# 추가될 호선의 역
choice

,지하철역,주소,위도,경도,호선명,후보역
27,광나루(장신대),서울 광진구 광장동 237,37.545299,127.103521,5호선,강변(동서울터미널)
195,아차산(어린이대공원후문),서울 광진구 능동 256-16,37.552249,127.089565,5호선,강변(동서울터미널)
136,삼전,서울 송파구 잠실동 347,37.504549,127.087364,9호선,일원
98,망우,서울 중랑구 상봉동 172,37.599296,127.092367,중앙선,봉화산(서울의료원)


In [30]:
dist = []
for i in candidate_location_copy.index:
    station = candidate_location_copy.loc[i]
    can_gps = station['위도'], station['경도']
    for j in choice[choice['후보역'] == station['지하철역']].index:
        choice_gps = choice.loc[j, '위도'], choice.loc[j, '경도']
        dist.append((station['지하철역'], choice.loc[j,'지하철역'], distance.distance(can_gps, choice_gps).km))

In [31]:
# 연결 하고자 하는 역이 여러개인 경우 거리 짧은 것 뽑기
dist = pd.DataFrame(dist, columns=['지하철역', '연결역', '거리'])

choice['거리'] = list(dist['거리'])

choice = choice.groupby('후보역').min().reset_index()

choice

,후보역,지하철역,주소,위도,경도,호선명,거리
0,강변(동서울터미널),광나루(장신대),서울 광진구 광장동 237,37.545299,127.089565,5호선,1.370774
1,봉화산(서울의료원),망우,서울 중랑구 상봉동 172,37.599296,127.092367,중앙선,2.005859
2,일원,삼전,서울 송파구 잠실동 347,37.504549,127.087364,9호선,2.299789


In [32]:
## STEP 6 ##

map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

# 환승 후보역 지도에 표시
for i in candidate_location_copy.index: 
    folium.Marker(
        radius=1000, 
        location=[candidate_location_copy.위도[i],candidate_location_copy.경도[i]],
        popup=folium.Popup(candidate_location_copy.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        icon = folium.Icon(
            color='purple',
            icon='train-subway',
            prefix='fa'
        )
    ).add_to(map)
    
for i in candidate_location_copy.index: 
    folium.Circle(
        radius=2300, 
        location=[candidate_location_copy.위도[i],candidate_location_copy.경도[i]],
        popup=folium.Popup(candidate_location_copy.호선명[i], max_width=200),
        tooltip=candidate_location_copy.지하철역[i]+'(환승후보역)',
        color='red',
        fill=True,
        fill_opacity=0.2,
        fill_color='red'
    ).add_to(map)
    

for i in choice.index:
    folium.Circle(
    radius=200, 
    location=[choice.위도[i],choice.경도[i]],
    popup=folium.Popup(choice.호선명[i], max_width=200),
    tooltip=choice.지하철역[i],
    color='yellow',
    fill=True,
    fill_color='yellow'
).add_to(map)
    
    
title='<h3 align="center" style="font-size:30px">STEP 6: 최종 환승 후보역 </h3>'
map.get_root().html.add_child(folium.Element(title))

map

c:\Users\user\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [33]:
choice[['후보역', '지하철역', '거리']].to_csv('../data/수정 후 데이터/최종환승역.csv', index=False)